LDA Pipeline (SQL->LDA)

In [2]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import similarities

In [7]:
#Read orig text and preproc from SQL db
#CONNECT TO SQL
#import mysql

import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb

sql_db = MySQLdb.connect(host='127.0.0.1',user='covidAnalysis', password = "k34p63MbDDcZ9yf4",db='covid19framing')
cursor = sql_db.cursor()

#originaltext = [item[1].decode() for item in results]


In [6]:
cursor.execute("select * from twitter where text_preproc is not NULL")
results = cursor.fetchall()
len(results)




1440287

In [4]:
results[2]

(5528,
 None,
 '1220397141308465153',
 'Pritzker people ran/controlled the #COVID19 pandemic in Wuhan &amp; Illinois! https://t.co/hPgTygaR1W',
 20,
 3,
 19,
 datetime.datetime(2022, 10, 31, 23, 56, 4),
 datetime.datetime(2023, 3, 7, 10, 58, 11),
 None,
 None,
 1587231966721675264,
 1587231966721675264,
 'pritzker people ran hpgtygarw')

In [7]:
#Gets preprocessed and original texts (Checked and they are good)
doclist = [item[-1].split() for item in results] #preprocessed 
originaltext = [item[3] for item in results] #originaltext

In [8]:
def prepare_corpus(doc_clean,no_below=5,no_above=0.5):
	'''
	Reutrns: A dictionary of the final set of terms and document-term frequency matrix

	# adapted from https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

	parameter doc_clean: processed set of documents (type: string)
	parameter no_below: exclude words that only appear $no_below$ times or less in the whole corpus
	parameter no_above: any words included in more than $no_above$ percentage of documents will be excluded

	'''
	# Creating the term dictionary of our courpus, where every unique term is assigned an index.
	dictionary = corpora.Dictionary(doc_clean)

	dictionary.filter_extremes(no_below=no_below, no_above=no_above)
	# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
	doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
	# generate LDA model
	return dictionary,doc_term_matrix


In [9]:
#Prepare Corpus
vocab_dict, doc_term_matrix = prepare_corpus(doclist)

In [10]:
import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

/bin/bash: apt-get: command not found
openjdk version "1.8.0_362-ea"
OpenJDK Runtime Environment (build 1.8.0_362-ea-b01)
OpenJDK 64-Bit Server VM (build 25.362-b01, mixed mode)


In [11]:
#os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '../../mallet-2.0.8/bin/mallet' 

In [12]:
#LDA

import os
from gensim.models.wrappers import LdaMallet

n_topics = 15; iterations = 200
ldaMallet = LdaMallet(mallet_path=mallet_path, corpus=doc_term_matrix, num_topics=n_topics, id2word=vocab_dict,iterations=iterations,random_seed = 54321,optimize_interval=50)


Mallet LDA: 15 topics, 4 topic bits, 1111 topic mask
Data loaded.
max tokens: 76
total tokens: 20482703
<10> LL/token: -11.19087
<20> LL/token: -10.95016
<30> LL/token: -10.47281
<40> LL/token: -10.15202

0	3.33333	coronavirus community pandemic lockdown access people health care continues things left millions world affected covid_ support quarantine global narendramodi climate 
1	3.33333	china health shanghai people city local coronavirus testing pandemic chinese omicron news vaccination national test asymptomatic beijing hongkong tests today 
2	3.33333	omicron coronavirus week students health pandemic state testing cases schools people government variant test days positive today tests case wave 
3	3.33333	cases deaths total india coronavirus hours today reported active rate update reports death positive number positivity recoveries tests delhi fresh 
4	3.33333	people coronavirus pandemic longcovid uk wearamask health covidisnotover kids vaccines borisjohnson world save care nhs johns

In [17]:
import numpy as np

In [18]:
doc_topics = get_doc_topics(ldaMallet,originaltext,n_topics,len(doc_term_matrix),top_doc_n=20,show_top_doc=True)
doc_topics

Topic  0  :  [('pandemic', 0.04003369721550237), ('global', 0.01630581531831883), ('support', 0.015220230452584188), ('response', 0.013139296518964672), ('impact', 0.010729105246626146), ('community', 0.010018240029977572), ('countries', 0.00952022108459296), ('workers', 0.009517465793055563), ('access', 0.00917994257972436), ('crisis', 0.008438080333280064)]
top docs: 

[0.49393939393939396, '@BeeAsMarine @ishdeep_kohli @ConnectAID_int @AmandaRay02 @Sushmita4Rights @GaelleMogli @FrantzJazzy @JoinIsmail @GiliianSmith @FAO @UNESCO @Sdg13Un @Faypumping @AdamRogers2030 @salhaj @BagalueSunab @aippnet @UN4Indigenous @Cop27P @UNBiodiversity @GlobalGoalsUN @SDGaction @DrSamira_Asma @MoetiTshidi @UNDPClimate @RoseMwebaza @cfleddrdc @UNFCCC @UNFCCC_CTCN @WECF_INT @SDGAdvocates @drtlaleng @mentorafrika @Brian_J_Keane @FAOclimate @srpoverty @Momentum_UNFCCC @JanetRoganCOP26 @AllegraCOP26 @COP26 @FordFoundation @esmeraldadereth @Cshells33Wells @andersen_inger By @Lepariyo #IamSDGAmbassador Can glo

array([[0.06770833, 0.05208333, 0.05208333, ..., 0.06770833, 0.06770833,
        0.09895833],
       [0.11737089, 0.04694836, 0.04694836, ..., 0.04694836, 0.06103286,
        0.07511737],
       [0.06289308, 0.06289308, 0.06289308, ..., 0.06289308, 0.08176101,
        0.06289308],
       ...,
       [0.06666667, 0.08205128, 0.06666667, ..., 0.06666667, 0.05128205,
        0.06666667],
       [0.05649718, 0.05649718, 0.05649718, ..., 0.07344633, 0.09039548,
        0.05649718],
       [0.04761905, 0.07619048, 0.07619048, ..., 0.06190476, 0.06190476,
        0.07619048]])

In [13]:
def get_doc_topics(model,docs,n_topics,doc_number,top_doc_n=10,show_top_doc=False):
  '''
  Computes and shows doc-topic distribution
  -----------------------------------------
  parameters:
  -----------------
  model: A LdaMallet gensim wrapper
  n_topics: number of topics that the model was trained with
  doc_number: number of documents in total
  top_doc_n: number of top documents to show for each topic
  show_top_doc: whether to show top documents for each topic or not

  returns:
  -----------------
  doc_topics_np: a numpy array of size (doc,n_topics) that shows the distribution of topics for each doc
  '''
  #test something
  doc_topics = model.load_document_topics() #loading doc-topic distribution
  doc_topics_np = np.zeros((doc_number,n_topics)) #initializing a numpy array to collect all topic-doc matrices

  docc = 0
  #reading one by one from doc_topics LDA output
  for D in doc_topics:
    doc_topics_np[docc,:] = np.asarray(D)[:,1]
    docc = docc + 1
  #top_doc_n = 10

  if show_top_doc:
    #printing top documents of each topics
    for i in range(doc_topics_np.shape[1]):
      top_doc = np.argsort(doc_topics_np[:,i])[-top_doc_n:]
      print('Topic ', i,' : ',model.show_topic(i))
      print('top docs: \n')
      for ind in reversed(range(len(top_doc))):
        print([doc_topics_np[top_doc[ind],i],docs[top_doc[ind]]])
        print("........----------------........")
      print("------------------------------------------------------------------")
      print('\n\n\n')

  return doc_topics_np